# **Análise Exploratória dos Hábitos de Jogo dos Jogadores de Valorant**

## Configurações Iniciais

### Imports

In [95]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, expr, count, sum, max, udf, dayofweek, date_format, when, mean, median
from pyspark.sql.types import StringType
sys.path.append('../src/')
from aws.aws import Aws
import io
import pandas as pd 
import boto3
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import warnings
warnings.filterwarnings("ignore")

### Criando a seção Spark

In [96]:
spark = SparkSession.builder.appName("ValorantDataAnalysis").getOrCreate()

In [97]:
spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.hadoop.fs.s3a.access.key", os.getenv('AWS_ACCESS_KEY_ID'))
spark.conf.set("spark.hadoop.fs.s3a.secret.key", os.getenv('AWS_SECRET_ACCESS_KEY'))

### Config pandas

In [98]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Configurar o formato dos números
pd.set_option('display.float_format', '{:.2f}'.format)

### Instânciando as classe utilizadas.

In [99]:
aws = Aws()

## Carregando os dataframes.

### Utils.

In [100]:
def get_files(bucket_name : str, folder_path : str) -> list:
    """"""
    objects = aws.list_objetcs_s3(bucket_name, folder_path)


    return objects

def concat_files_s3(objects):
    """"""

    json_files = [obj['Key'] for obj in objects]

    for file in json_files:

        response = aws.read_s3_v2(bucket_name='s3-tcc-fia-valorant', folder_path=file)
        json_data = response['Body'].read().decode('utf-8')

    return io.StringIO(json_data)

def read_spark(data_io):
    """"""
    data_io = pd.read_csv(data_io)
    return spark.createDataFrame(data_io)

def create_dataframe(bucket_name : str, folder_path : str):
    """"""
    objects = get_files(bucket_name, folder_path)
    data_io = concat_files_s3(objects)
    df = read_spark(data_io)

    return df

def save_dataframe_csv(bucket_name, folder_path, file_name, data, file_format):
    # Convert DataFrame to CSV string
    csv_buffer = io.StringIO()
    data.toPandas().to_csv(csv_buffer, index=False)

    # Retrieve CSV data from buffer
    csv_buffer_value = csv_buffer.getvalue()

    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = file_name + '_' + date + file_format
    file_path = folder_path + file_name

    # Write CSV string to S3
    s3 = boto3.resource('s3')

    try:
        s3.Object(bucket_name, file_path).put(Body=csv_buffer_value)
        print(f"Data was written to S3://{bucket_name}/{folder_path}")

    except Exception as e:

        print(f"Error: {e}")
    
        return False



### df_player_book

In [101]:
df_player_book = create_dataframe('s3-tcc-fia-valorant', 'valorant/refined/player-book/')

## EDA

### Filtros

In [102]:
total_matches_before = df_player_book.select('match_id').distinct().count()
df_player_book = df_player_book.dropna()
total_matches_after = df_player_book.select('match_id').distinct().count()
total_matches_removed = total_matches_before - total_matches_after
print(f"Total de partidas antes da remoção dos valores nulos: {total_matches_before}")
print(f"TTotal de partidas depois da remoção dos valores nulos: {total_matches_after}")
print(f"Total matches removidas: {total_matches_removed}")

Total de partidas antes da remoção dos valores nulos: 298
TTotal de partidas depois da remoção dos valores nulos: 218
Total matches removidas: 80


In [103]:
total_matches_before = df_player_book.select('match_id').distinct().count()
df_player_book = df_player_book.where((col("total_players_match") == 10) & (col("total_players_team") == 5))
total_matches_after = df_player_book.select('match_id').distinct().count()
total_matches_removed = total_matches_before - total_matches_after
print(f"Total de partidas antes de aplicar o filtro de quantidade de jogadores: {total_matches_before}")
print(f"Total de partidas depois de aplicar o filtro de jogadores: {total_matches_after}")
print(f"Total de partidas removidas: {total_matches_removed}")

Total de partidas antes de aplicar o filtro de quantidade de jogadores: 218
Total de partidas depois de aplicar o filtro de jogadores: 212
Total de partidas removidas: 6


### Partidas

In [221]:
df_player_book_pd = df_player_book.toPandas()
df_player_book_pd_distinct = df_player_book_pd.drop_duplicates(subset=['match_id'])

In [ ]:
df_matches_raw_describe = df_player_book_pd.describe()
df_matches_raw_describe_pivot = pd.pivot_table(df_matches_raw_describe, columns=["count", 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
df_matches_raw_describe_pivot

#### Dias

In [142]:
weekDayOrdered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

matches_per_day = (
    
                    df_player_book_pd_distinct.groupby([df_player_book_pd_distinct.date_match, df_player_book_pd_distinct.week_day])['match_id']
                    .count()
                    .reset_index(name='count')
                    .reset_index(drop = True)
                    )

means_time_matches = df_player_book_pd_distinct[['match_id', 'week_day', 'playtime_minutes_value']]
means_time_matches = (
                        means_time_matches.groupby('week_day')['playtime_minutes_value']
                        .mean()
                        .reindex(weekDayOrdered)
                        .reset_index(name = 'mean')
                        .reset_index(drop = True)                     
                        .round(2)
)

mean_matches_per_days_week = (
                                matches_per_day.groupby(matches_per_day.week_day)['count']
                                .mean()
                                .reindex(weekDayOrdered) 
                                .reset_index(name = 'mean')
                                .reset_index(drop = True)
                                .round(3)
                                )

mean_matches_per_days_week = (
                                matches_per_day.groupby(matches_per_day.week_day)['count']
                                .mean()
                                .reindex(weekDayOrdered) 
                                .reset_index(name = 'mean')
                                .reset_index(drop = True)
                                .round(3)
                                )

top_seven_days_matches = (
    df_player_book_pd_distinct.groupby(['date_match', 'week_day'])['week_day']
    .count()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .reset_index(drop=True)
    .head(5)
)

fig1 = px.bar(x = matches_per_day['date_match'], y = matches_per_day['count'], title= 'Numero de partidas por dia.')
fig1.update_layout(xaxis_title='Dias', yaxis_title='Quantidade de partidas')
fig1.show()

fig2 = px.line(means_time_matches, x = 'week_day', y = 'mean', title = 'Média de tempo jogado por dia da semana.')
fig2.update_layout(xaxis_title='Dias da semana', yaxis_title='Tempo médio em minutos')
fig2.show()

fig3 = px.line(mean_matches_per_days_week, x = 'week_day', y = 'mean', title = 'Média de partidas por dia.')
fig3.update_layout(xaxis_title='Dias da semana', yaxis_title='Média de partidas por dia')
fig3.show()

fig4 = px.bar(top_seven_days_matches, x='week_day', y='count', color='count',
             height=400, title='Top sete dias por quantidade de partidas.')
fig4.update_xaxes(type='category')  # Define o tipo de dados da coluna week_day como categórico
fig4.update_layout(xaxis_title='Dias da semana', yaxis_title='Quantidade de partidas')
fig4.show()

A análise exploratória revelou números significativos sobre os hábitos de jogo dos jogadores de Valorant. Ao longo da semana, os jogadores realizam em média **1,63 a 2,22 jogos por dia**. Essa variação é evidente ao observar as médias de jogos por dia da semana:

**Quartas-feiras, quintas-feiras e sábados** registram cerca de **1,8 jogos**.
**Segundas e sextas-feiras** têm aproximadamente **1,73** a **1,80 jogos**.
As **terças-feiras** apresentam a **média mais baixa**, com **1,75 jogos**.
No entanto, os **domingos se destacam** com a **média mais alta** de **2,22 jogos**.
Além disso, é importante ressaltar alguns dias específicos com números expressivos de jogos. O dia **19/12/2022 (segunda-feira)** e o dia **01/05/2022 (domingo)** registraram um total de **5 jogos cada**. Já os dias **12/05/2023 (sexta-feira) e 06/05/2023 (sábado)** contabilizaram **4 jogos cada**.

**CURIOSIDADE:** As **quinta-feira** é o dia com a **menor média** de jogos, porém tem o **maior tempo** por partida. Ainda não sei ao certo a corelação mas veremos nas analíses futuras.

Esses números destacados indicam momentos de maior participação dos jogadores, **sugerindo a ocorrência de eventos especiais**, torneios ou outras atividades que estimularam a **intensificação do engajamento dos jogadores**.

Esses insights podem orientar a **alocação de recursos e a implementação de iniciativas** direcionadas para maximizar o engajamento dos jogadores de Valorant.

#### Plantio e desarmamento da spike

In [145]:
matches_plat_defuse = (
                        df_player_book_pd_distinct.groupby(['result'])[['plants_value', 'defuses_value']]
                        .sum()
                        .sort_values('plants_value', ascending = False)
                        .reset_index()
)

fig1 = px.bar(matches_plat_defuse, x = 'result', y = 'plants_value', text_auto='.2s',
            title="Quantidade de Plantio por Resultado", color = 'result')
fig1.update_layout(xaxis_title='Resultado da partida', yaxis_title='Quantidade de plantio')
fig1.show()

fig2 = px.bar(matches_plat_defuse, x = 'result', y = 'defuses_value', text_auto='.2s',
            title="Quantidade de Desarmes por Resultado", color = 'result')
fig2.update_layout(xaxis_title='Resultado', yaxis_title='Quantidade de desarmamento')
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Box(y=matches_plat_defuse['plants_value'],
                    name='Quantidade de plantio'))

fig3.add_trace(go.Box(y=matches_plat_defuse['defuses_value'],
                    name='Quantidade de desarmamento'))
fig3.update_layout(title="Distribuição de Plantio e Desarmes por Resultado")
fig3.show()

Em média, **equipes vitoriosas** plantaram a spike **135 vezes** e desarmaram **37 vezes**.
Equipes que **sofreram derrotas** plantaram a spike **130 vezes** e desarmaram **46 vezes**.
**Em empates**, ocorreram apenas **6 plantios da spike** e **nenhum** desarme.

Esses dados destacam a **importância dos plantio e desarmes** para o resultado das partidas de Valorant. Equipes com um **alto número de plantas e desarmes** têm uma **maior probabilidade de alcançar a vitória**. Portanto, é crucial para os jogadores e equipes desenvolverem estratégias eficientes para essas ações dentro do jogo.

#### Mapas

In [146]:
maps_matches = df_player_book_pd_distinct[['match_id', 'map_name']]
maps_matches = (
                maps_matches.groupby('map_name')['match_id']
                .count()
                .reset_index(name='count')
                .reset_index(drop=True)
                .sort_values('count', ascending = True)

)

maps_matches_result = df_player_book_pd_distinct[['match_id', 'map_name', 'result']]
maps_matches_result = (
                maps_matches_result.groupby(['map_name', 'result'])['match_id']
                .count()
                .reset_index(name='count')
                .sort_values('count', ascending=False)
)

fig1 = px.bar(maps_matches, x = 'map_name', y = 'count', text_auto='.2s',
            title="Quantidade de partidas por mapa", color = 'count')
fig2.update_layout(xaxis_title='Nome do mapa', yaxis_title='Quantidade de partidas')
fig2.show()

fig2 = px.bar(maps_matches_result, x='map_name', y='count', color='result',
              title='Resultado de Partidas por Mapa', barmode='group')
fig3.update_layout(xaxis_title='Nome do mapa', yaxis_title='Quantidade por resultado')
fig3.show()

fig4 = px.sunburst(data_frame=maps_matches_result, path=['result', 'map_name'], values='count',
                  title='Mandala: Resultados das Partidas por Mapa')
fig4.update_traces(textinfo='label+percent entry')
fig4.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig4.show()

Observa-se que o mapa **Haven** possui o **maior número** de partidas, com um total de **39**. Em seguida, temos os mapas **Ascent** e **Bind** com **37 partidas**, respectivamente. Por outro lado, os mapas **Breeze** e **Fracture**, ambos com **18 partidas**, apresentam uma quantidade menor de partidas registradas.

Em relação às vitórias, podemos observar que os mapas **Bind** e **Haven** se destacam, representando cada um **23%** das vitórias totais. Esses mapas são **particularmente favoráveis** para as equipes, resultando em um número **significativo de vitórias**. Além disso, o mapa **Ascent** também apresenta um desempenho sólido, contribuindo com **22%** das vitórias.

Por outro lado, quando analisamos as **derrotas**, o mapa **Ascent** chama a atenção, com **15** ocorrências. Isso representa uma parcela considerável de **29%** das **derrotas** totais. Os mapas **Bind** e **Haven** também possuem uma quantidade **significativa de derrotas**, com **14** cada, representando **27%** das **derrotas**.

Em relação aos empates, eles são menos comuns, mas é interessante observar que o mapa **Haven** se destaca, representando **40%** dos **empates** registrados.

Os dados ressaltam a importância dos diferentes mapas no desempenho das equipes. Estratégias adequadas para cada mapa podem influenciar diretamente os resultados das partidas. É essencial que os jogadores e equipes considerem essas informações ao planejar suas táticas e competir no futuro.

Vale lembrar que em Valorant a **Riot** tenta garantir uma **distribuição igualitaria** entre os mapas como podemos ver nos mais antigos.

#### Rounds

In [149]:
rounds_matches = df_player_book_pd_distinct[['match_id', 'total_rounds', 'result']]
rounds_matches_agg = (
                rounds_matches.groupby('match_id')['total_rounds']
                .max()
                .reset_index(name='max')
                .reset_index(drop=True)
                .sort_values('max', ascending = False)
                .head(10)

)

fig1 = px.bar(rounds_matches_agg, x='match_id', y='max', title='Top 10 partidas com maior numero de rounds.')
fig1.update_layout(xaxis_title='Id da partida', yaxis_title='Mumero de rounds')
fig1.show()

fig2 = px.histogram(df_player_book_pd_distinct, x='total_rounds', color="result", barmode="group", title='Distribuição da Quantidade de Rounds por Partida')
fig2.update_layout(xaxis_title='Quantidade de Rounds', yaxis_title='Frequência')
fig2.show()


# Visualização dos resultados das partidas por mapa
fig3 = px.sunburst(data_frame=rounds_matches, path=['total_rounds', 'result', 'match_id'], 
                   values='total_rounds', title='Mandala: Resultados de partidas por numero de rounds.')
fig3.update_traces(textinfo='label+percent entry')
fig3.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig3.show()


Ao analisar os dados, podemos observar que a quantidade de rodadas jogadas **varia consideravelmente**, com um mínimo de **13 rodadas** e um máximo de **30 rodadas**. A maioria das partidas parece ter sido jogada em torno de **19** a **24 rodadas**, com algumas exceções de partidas mais curtas ou mais longas.

Em relação aos resultados das partidas, temos que a **vitória** ocorreu em aproximadamente **54%** das vezes, enquanto a **derrota** ocorreu em cerca de **46% das partidas**. Essa proporção indica que há um certo **equilíbrio** entre as **vitórias** e **derrotas** nas partidas registradas.

Como podemos observar no plot de **agrupamento de resultados por quantidade de rounds por partidas** apartir do **round 22** **não** temos a **definição** do resultado de uma partida nos **rounds impares** subsequentes, pois **apartir** do do **round 22** todos os rounds **impares** são os **match points (round de desempate)**.

### Jogador

##### Quantidade de partidas

In [165]:
matches_per_player_top10 = (
                        df_player_book_pd.groupby(['platform_user_identifier'])['platform_user_identifier']
                        .count()
                        .reset_index(name = 'count')
                        .reset_index(drop = True)
                        .sort_values('count', ascending = False)
                        .head(10)
                        )



fig1 = px.bar(matches_per_player_top10, x = 'platform_user_identifier', y = 'count', title = 'Top 10 numero de partidas por jogador.')
fig1.show()

Observa-se que a contagem de partidas para o jogador **RayzenSama#6999** é significativamente **maior em comparação aos demais** jogadores da lista, devido ao **processo de aquisição dos dados** estar amarrado ao ID desse jogador. Isso pode afetar a representatividade dos resultados em relação a outros jogadores.
É importante ressaltar que a quantidade de partidas está relacionada à **proximidade com o jogador RayzenSama#6999**. Dessa forma, jogadores com uma **contagem maior** de partidas têm uma maior **relação ou interação** com o jogador **RayzenSama#6999** no contexto da amostra analisada.

##### Econômia

In [237]:
player_econ = df_player_book_pd[['match_id', 'result','platform_user_identifier', 'total_spent_credits', 'econ_rating_value']]

econ_mean = player_econ['total_spent_credits'].mean()
econ_median = player_econ['total_spent_credits'].median()
econ_max = player_econ['total_spent_credits'].max()
econ_min = player_econ['total_spent_credits'].min()

econ_rating_mean = player_econ['econ_rating_value'].mean()
econ_rating_median = player_econ['econ_rating_value'].median()
econ_rating_max = player_econ['econ_rating_value'].max()
econ_rating_min = player_econ['econ_rating_value'].min()


victory_mean = player_econ[player_econ['result'] == 'victory']['total_spent_credits'].mean()
defeat_mean = player_econ[player_econ['result'] == 'defeat']['total_spent_credits'].mean()

victory_median = player_econ[player_econ['result'] == 'victory']['total_spent_credits'].median()
defeat_median = player_econ[player_econ['result'] == 'defeat']['total_spent_credits'].median()

victory_max = player_econ[player_econ['result'] == 'victory']['total_spent_credits'].max()
defeat_max = player_econ[player_econ['result'] == 'defeat']['total_spent_credits'].max()

victory_min = player_econ[player_econ['result'] == 'victory']['total_spent_credits'].min()
defeat_min = player_econ[player_econ['result'] == 'defeat']['total_spent_credits'].min()

player_econ = df_player_book_pd.sort_values(by='econ_rating_value', ascending=False)
top_10_players = df_player_book_pd.head(10)

def custom_mean(values):
    numeric_values = pd.to_numeric(values, errors='coerce')
    return numeric_values.mean()
grouped_df = df_player_book_pd.groupby(['econ_rating_value', 'result', 'platform_user_identifier']).agg(custom_mean).reset_index()
sorted_df = grouped_df.sort_values(by='econ_rating_value', ascending=False)
top_10_players = sorted_df.head(10)


fig1 = make_subplots(rows=1, cols=2)

# Adicionando os gráficos de barras ao subplot
fig1.add_trace(go.Bar(name='Média total gasto', x=['Econ'], y=[econ_mean]), row=1, col=1)
fig1.add_trace(go.Bar(name='Mediana total gasto', x=['Econ'], y=[econ_median]), row=1, col=1)
fig1.add_trace(go.Bar(name='Máximo total gasto', x=['Econ'], y=[econ_max]), row=1, col=1)
fig1.add_trace(go.Bar(name='Mínimo total gasto', x=['Econ'], y=[econ_min]), row=1, col=1)

fig1.add_trace(go.Bar(name='Média rating econômico', x=['Econ Rating'], y=[econ_rating_mean]), row=1, col=2)
fig1.add_trace(go.Bar(name='Mediana rating econômico', x=['Econ Rating'], y=[econ_rating_median]), row=1, col=2)
fig1.add_trace(go.Bar(name='Máximo rating econômico', x=['Econ Rating'], y=[econ_rating_max]), row=1, col=2)
fig1.add_trace(go.Bar(name='Mínimo rating econômico', x=['Econ Rating'], y=[econ_rating_min]), row=1, col=2)

# Personalização do layout
fig1.update_layout(title='Estatísticas Econômicas',
                  xaxis_title='Estatísticas',
                  yaxis_title='Valor')

# Exibição do gráfico
fig1.show()

fig2 = make_subplots(rows=2, cols=2, subplot_titles=("Média", "Mediana", "Máximo", "Mínimo"))
fig2.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_mean, defeat_mean]), row=1, col=1)
fig2.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_median, defeat_median]), row=1, col=2)
fig2.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_max, defeat_max]), row=2, col=1)
fig2.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_min, defeat_min]), row=2, col=2)
fig2.update_layout(height=400, width=1166, title_text="Estatísticas do valor total gasto por resultado")
fig2.show()

fig3 = px.scatter(player_econ, x='avg_spent_credits', y='avg_econ_rating_value', color='result', labels={'avg_spent_credits': 'Valor Médio de Economia', 'avg_econ_rating_value': 'Rating Econômico'}, title='Relação entre Valor Médio de Economia e Rating Econômico', trendline='ols')
fig3.show()

fig4 = go.Figure(data=go.Bar(
    x=top_10_players['platform_user_identifier'],
    y=top_10_players['econ_rating_value'],
    text=top_10_players['econ_rating_value'],
    textposition='auto',
))

fig4.update_layout(
    title='Top 10 Jogadores - Melhores em Economia',
    xaxis_title='Jogador',
    yaxis_title='Econ Rating',
)
fig4.show()

fig5 = px.bar(top_10_players, x='platform_user_identifier', y='econ_rating_value', color='result',
             title='Top 10 Jogadores - Eco Rating e Resultados')
fig5.show()



Dentre as várias estratégias envolvidas em uma partida de valorant, as **estatísticas econômicas** desempenham um papel crucial na **tomada de decisões** dos jogadores.

Ao analisar os dados, podemos observar que, em **média**, os jogadores **gastaram 50.061,87 mil créditos** durante a partida. Esses créditos são utilizados para **adquirir armas, equipamentos e habilidades especiais**, proporcionando **vantagens táticas e aumentando as chances de sucesso**.

A **mediana dos gastos econômicos**, situada em **49.950 mil créditos**, indica que **metade dos jogadores** gastou **menos** do que esse valor, enquanto a outra **metade** gastou mais.

Além disso, é **interessante** observar o **valor máximo de gastos econômicos** registrado na partida, alcançando **127.650 mil créditos**. Isso sugere que alguns jogadores **optaram por investir em melhorias substanciais**, como **armas de alto calibre** ou **itens especiais**, na tentativa de obter uma **vantagem decisiva** sobre seus oponentes.

Por outro lado, o valor **mínimo de gastos econômicos** foi de **4.200 mil créditos**, indicando que alguns jogadores **optaram por uma abordagem mais econômica**, talvez priorizando outras estratégias ou reservando recursos para **momentos-chave**, como **ajudar** um colega de equipe **doando equipamentos**.

Além das estatísticas econômicas, o **rating de econômia** também desempenha um **papel importante**.

Com um valor **médio de 57.43605**, esse rating reflete o **desempenho dos jogadores** em termos de **gerenciamento financeiro**. Um **rating mais elevado indica uma eficiente utilização dos recursos** disponíveis, possibilitando um **maior investimento em melhorias e equipamentos** durante a partida.

Já o valor **mínimo de 9** sugere que alguns jogadores **tiveram dificuldades** em equilibrar seus gastos, o que pode **impactar diretamente** seu **desempenho** durante as batalhas.

As analíses também apresenta informações sobre o **valor total gasto por resultado (vitória e derrota)**.
Em **média**, os jogadores **vitoriosos gastaram** cerca de **50.153,06 mil créditos**, enquanto aqueles que **foram derrotados** apresentaram uma **média de gastos** de **48.626,85 mil créditos**.

Esses números indicam que, apesar de uma ligeira diferença, **não há** uma **discrepância significativa** nos gastos entre as **vitórias e as derrotas**. Isso sugere que o **sucesso** ou **fracasso** na partida **depende de outros fatores** além dos gastos econômicos, como **habilidades individuais**, estratégias de equipe e tomadas de decisões táticas.

Em resumo, as **estatísticas econômicas, o rating de economia e os valores de gastos por resultado** fornecem **insights valiosos** sobre o jogo Valorant. Esses dados refletem as escolhas e estratégias dos jogadores, permitindo uma análise mais aprofundada do desempenho individual e coletivo durante a partida. Com base nessas **informações**, os jogadores podem **ajustar** suas **táticas, otimizar seus gastos** e buscar uma vantagem competitiva rumo à vitória.

##### Rank

In [306]:
player_rank = df_player_book_pd[['platform_user_identifier', 'rank_value', 'current_rank_value', 'date_match', 'rank_number_value', 'current_rank_number_value']]

top_players = player_rank.groupby('platform_user_identifier')['current_rank_number_value'].max().reset_index()
bottom_players = player_rank.groupby('platform_user_identifier')['current_rank_number_value'].min().reset_index()

top_players = top_players.sort_values('current_rank_number_value', ascending=False).head(10)
bottom_players = bottom_players.sort_values('current_rank_number_value', ascending=True).head(100)

player_rank['rank_change'] = player_rank['current_rank_number_value'] - player_rank['rank_number_value']
player_rank['rank_movement'] = player_rank['rank_change'].apply(lambda x: 'Progressão' if x > 0 else ('Regressão' if x < 0 else 'Manutenção'))
rank_movement_counts = player_rank['rank_movement'].value_counts()
rank_movement_counts = rank_movement_counts.sort_values(ascending=False)
rank_movement_percentages = rank_movement_counts / rank_movement_counts.sum() * 100

text_labels = [f'{value:.1f}%' for value in rank_movement_percentages]

rank_counts = player_rank['current_rank_value'].value_counts()

fig1 = make_subplots(rows=2, cols=1, subplot_titles=("Top 10 Melhores Jogadores por Rank", "Top 10 Piores Jogadores por Rank"))

fig1.add_trace(
    go.Bar(
        x=top_players['platform_user_identifier'],
        y=top_players['current_rank_number_value'],
        marker=dict(color='blue'),
    ),
    row=1, col=1
)

fig1.add_trace(
    go.Bar(
        x=bottom_players['platform_user_identifier'],
        y=bottom_players['current_rank_number_value'],
        marker=dict(color='red'),
    ),
    row=2, col=1
)

fig1.update_layout(
    height=800,
    showlegend=False,
    title_text="Top 10 jogadores por Rank",
)
fig1.update_xaxes(title_text="Jogador", row=2, col=1)
fig1.update_yaxes(title_text="Rank", row=1, col=1)
fig1.update_yaxes(title_text="Rank", row=2, col=1)
fig1.show()

fig2 = px.sunburst(data_frame=df_player_book_pd, path=['current_rank_value', 'rank_value'], values='rank_number_value',
                  title='Mandala: Resultados das Partidas por Mapa')
fig2.update_traces(textinfo='label+percent entry')
fig2.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig2.show()

fig3 = go.Figure(data=go.Bar(
    x=rank_movement_counts.index,
    y=rank_movement_counts.values,
    text=text_labels,
    marker=dict(color='blue'),
))

fig3.update_layout(
    title='Movimento de Rank dos Jogadores',
    xaxis_title='Movimento de Rank',
    yaxis_title='Contagem',
)
fig3.show()

fig5 = go.Figure(data=go.Bar(
    x=rank_counts.index,
    y=rank_counts.values,
    marker=dict(color='blue'),
))

fig5.update_layout(
    title='Quantidade de Jogadores por Current Rank',
    xaxis_title='Current Rank',
    yaxis_title='Quantidade de Jogadores',
)

fig5.show()

No jogo Valorant, as informações do **rank na é fundamental** para a **qualidade técnica** das partidas. 

Com base nos gráficos, observamos que **58.3%** dos jogadores **progrediram em seus ranks**, representando um total de **1236 jogadores**. Isso indica que esses jogadores tiveram um **desempenho consistente** e **melhoraram suas habilidades**, o que resultou em um **avanço** em seus ranks. Por outro lado, **27.4% dos jogadores (581 jogadores) regrediram em seus ranks**, indicando que **enfrentaram dificuldades** e tiveram um desempenho **inferior** em relação a partidas anteriores. Além disso, **14.3% dos jogadores (302 jogadores)** mantiveram o mesmo rank, sugerindo uma **estabilidade em seu desempenho**.

Ao analisar a distribuição dos **jogadores por rank**, podemos identificar os ranks mais populares e menos populares. Entre os **ranks mais populares**, temos o **Bronze 3**, com **320 jogadores**, seguido pelo **Unranked**, com **274 jogadores**. Esses números indicam uma quantidade **significativa de jogadores** nessas faixas de rank. Por outro lado, ranks como **Diamond 1**, **Diamond 2**, **Iron 3**, e **Ascendant 2** apresentam uma **quantidade menor de jogadores**, variando de **25** a **6 jogadores**. O fato de haver **menos jogadores** nessas **faixas de rank** pode sugerir que esses ranks representam um **nível de habilidade mais alto** e exigem um desempenho **excepcional** para alcançá-los.

O movimento de rank dos jogadores mostra a **progressão, regressão ou estabilidade** em relação ao **desempenho individual**. Além disso, a distribuição dos jogadores por rank **fornece insights** sobre a **popularidade** e a **representatividade** de cada faixa de rank.

É importante destacar que essas informações podem variar de acordo com o **período analisado**.

##### Score

In [367]:
players_score = df_player_book_pd[['result', 'score_value', 'score_per_round_value', 'avg_score_value', 'avg_team_score_value', 'team_score_value']]

score_value_mean = players_score['score_value'].mean()
score_value_median = players_score['score_value'].median()
score_value_max = players_score['score_value'].max()
score_value_min = players_score['score_value'].min()

score_per_round_value_mean = players_score['score_per_round_value'].mean()
score_per_round_value_median = players_score['score_per_round_value'].median()
score_per_round_value_max = players_score['score_per_round_value'].max()
score_per_round_value_min = players_score['score_per_round_value'].min()

avg_score_value_mean = players_score['avg_score_value'].mean()
avg_score_value_median = players_score['avg_score_value'].median()
avg_score_value_max = players_score['avg_score_value'].max()
avg_score_value_min = players_score['avg_score_value'].min()

avg_team_score_value_mean = players_score['avg_team_score_value'].mean()
avg_team_score_value_median = players_score['avg_team_score_value'].median()
avg_team_score_value_max = players_score['avg_team_score_value'].max()
avg_team_score_value_min = players_score['avg_team_score_value'].min()

team_score_value_mean = players_score['team_score_value'].mean()
team_score_value_median = players_score['team_score_value'].median()
team_score_value_max = players_score['team_score_value'].max()
team_score_value_min = players_score['team_score_value'].min()

victory_mean = players_score[players_score['result'] == 'victory']['score_value'].mean()
defeat_mean = players_score[players_score['result'] == 'defeat']['score_value'].mean()

victory_median = players_score[players_score['result'] == 'victory']['score_value'].median()
defeat_median = players_score[players_score['result'] == 'defeat']['score_value'].median()

victory_max = players_score[players_score['result'] == 'victory']['score_value'].max()
defeat_max = players_score[players_score['result'] == 'defeat']['score_value'].max()

victory_min = players_score[players_score['result'] == 'victory']['score_value'].min()
defeat_min = players_score[players_score['result'] == 'defeat']['score_value'].min()

victory_mean = players_score[players_score['result'] == 'victory']['score_per_round_value'].mean()
defeat_mean = players_score[players_score['result'] == 'defeat']['score_per_round_value'].mean()

victory_median = players_score[players_score['result'] == 'victory']['score_per_round_value'].median()
defeat_median = players_score[players_score['result'] == 'defeat']['score_per_round_value'].median()

victory_max = players_score[players_score['result'] == 'victory']['score_per_round_value'].max()
defeat_max = players_score[players_score['result'] == 'defeat']['score_per_round_value'].max()

victory_min = players_score[players_score['result'] == 'victory']['score_per_round_value'].min()
defeat_min = players_score[players_score['result'] == 'defeat']['score_per_round_value'].min()

victory_mean = players_score[players_score['result'] == 'victory']['avg_score_value'].mean()
defeat_mean = players_score[players_score['result'] == 'defeat']['avg_score_value'].mean()

victory_median = players_score[players_score['result'] == 'victory']['avg_score_value'].median()
defeat_median = players_score[players_score['result'] == 'defeat']['avg_score_value'].median()

victory_max = players_score[players_score['result'] == 'victory']['avg_score_value'].max()
defeat_max = players_score[players_score['result'] == 'defeat']['avg_score_value'].max()

victory_min = players_score[players_score['result'] == 'victory']['avg_score_value'].min()
defeat_min = players_score[players_score['result'] == 'defeat']['avg_score_value'].min()

victory_median = players_score[players_score['result'] == 'victory']['team_score_value'].mean()
defeat_median = players_score[players_score['result'] == 'defeat']['team_score_value'].mean()

victory_median = players_score[players_score['result'] == 'victory']['team_score_value'].median()
defeat_median = players_score[players_score['result'] == 'defeat']['team_score_value'].median()

victory_max = players_score[players_score['result'] == 'victory']['team_score_value'].max()
defeat_max = players_score[players_score['result'] == 'defeat']['team_score_value'].max()

victory_min = players_score[players_score['result'] == 'victory']['team_score_value'].min()
defeat_min = players_score[players_score['result'] == 'defeat']['team_score_value'].min()

fig1 = sp.make_subplots(rows=2, cols=2, subplot_titles=("Pontuação", "Pontuação por round", "Média da pontuação", "Média da pontuação do time"))
fig1.add_trace(go.Bar(name='Pontuação Média', x=['Pontuação'], y=[score_value_mean]), row=1, col=1)
fig1.add_trace(go.Bar(name='Pontuação Mediana', x=['Pontuação'], y=[score_value_median]), row=1, col=1)
fig1.add_trace(go.Bar(name='Pontuação Máxima', x=['Pontuação'], y=[score_value_max]), row=1, col=1)
fig1.add_trace(go.Bar(name='Pontuação Minima', x=['Pontuação'], y=[score_value_min]), row=1, col=1)

fig1.add_trace(go.Bar(name='Pontuação por round Média', x=['Pontuação por Round'], y=[score_per_round_value_mean]), row=1, col=2)
fig1.add_trace(go.Bar(name='Pontuação por Round Mediana', x=['Pontuação por Round'], y=[score_per_round_value_median]), row=1, col=2)
fig1.add_trace(go.Bar(name='Pontuação por Round Máxima', x=['Pontuação por Round'], y=[score_per_round_value_max]), row=1, col=2)
fig1.add_trace(go.Bar(name='Pontuação por Round Mínima', x=['Pontuação por Round'], y=[score_per_round_value_min]), row=1, col=2)

fig1.add_trace(go.Bar(name='Pontuação Média(score/QtdRounds)', x=['Pontuação Média'], y=[avg_score_value_mean]), row=2, col=1)
fig1.add_trace(go.Bar(name='Pontuação Mediana (score/QtdRounds)', x=['Pontuação Média'], y=[avg_score_value_median]), row=2, col=1)
fig1.add_trace(go.Bar(name='Pontuação Máxima (score/QtdRounds)', x=['Pontuação Média'], y=[avg_score_value_max]), row=2, col=1)
fig1.add_trace(go.Bar(name='Pontuação Mínima (score/QtdRounds)', x=['Pontuação Média'], y=[avg_score_value_min]), row=2, col=1)

fig1.add_trace(go.Bar(name='Pontuação do time Média', x=['Pontuação média do time'], y=[avg_team_score_value_mean]), row=2, col=2)
fig1.add_trace(go.Bar(name='Pontuação do time Mediana', x=['Pontuação média do time'], y=[avg_team_score_value_median]), row=2, col=2)
fig1.add_trace(go.Bar(name='Pontuação do time Máxima', x=['Pontuação média do time'], y=[avg_team_score_value_max]), row=2, col=2)
fig1.add_trace(go.Bar(name='Pontuação do time Mínima', x=['Pontuação média do time'], y=[avg_team_score_value_min]), row=2, col=2)

fig1.update_layout(title='Estatísticas de Pontuação', xaxis_title='Estatísticas', yaxis_title='Pontuação')

fig1.show()

fig2 = make_subplots(rows=2, cols=2, subplot_titles=("Pontuação", "Pontuação por round", "Média da pontuação"))
fig2.add_trace(go.Scatter(x=players_score['score_value'], y=players_score['avg_team_score_value'], mode='markers', name='Pontuação vs. Pontuação média do time'), row=1, col=1)
fig2.add_trace(go.Scatter(x=players_score['score_per_round_value'], y=players_score['avg_team_score_value'], mode='markers', name='Pontuação por round vs. Pontuação média do time'), row=1, col=2)
fig2.add_trace(go.Scatter(x=players_score['avg_score_value'], y=players_score['avg_team_score_value'], mode='markers', name='Pontuação média vs. Pontuação média do time'), row=2, col=1)

fig2.update_layout(height=600, width=1165, title_text="Relação entre diferentes pontuações e a pontuação média do time")

fig2.show()

fig3 = make_subplots(rows=2, cols=2, subplot_titles=("Score Value", "Score per Round Value", "Average Score Value"))
fig3.add_trace(px.scatter(players_score, x='score_value', y='avg_team_score_value', labels={'score_value': 'Pontuação', 'avg_team_score_value': 'Pontuação média do time'}, title='Relação entre pontuação individual e do time', trendline='ols').data[0], row=1, col=1)
fig3.add_trace(px.scatter(players_score, x='score_per_round_value', y='avg_team_score_value', labels={'score_per_round_value': 'Pontuação por round', 'avg_team_score_value': 'Pontuação média do time'}, title='Relação entre pontuação por round e Pontuação média do time', trendline='ols').data[0], row=1, col=2)
fig3.add_trace(px.scatter(players_score, x='avg_score_value', y='avg_team_score_value', labels={'avg_score_value': 'Pontuação média', 'avg_team_score_value': 'Pontuação média do time'}, title='Relação entre pontuação média e pontuação média do time', trendline='ols').data[0], row=2, col=1)

fig3.show()

fig4 = make_subplots(rows=2, cols=2, subplot_titles=("Média", "Mediana", "Máximo", "Mínimo"))
fig4.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_mean, defeat_mean]), row=1, col=1)
fig4.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_median, defeat_median]), row=1, col=2)
fig4.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_max, defeat_max]), row=2, col=1)
fig4.add_trace(go.Bar(x=['Vitória', 'Derrota'], y=[victory_min, defeat_min]), row=2, col=2)

fig4.update_layout(height=400, width=1166, title_text="Estatísticas de pontuação por resultado")

fig4.show()

fig5 = px.scatter(players_score, x='avg_score_value', y='avg_team_score_value', color='result', labels={'avg_score_value': 'Pontuação Média', 'avg_team_score_value': 'Pontuação média do time'}, title='Relação entre pontuação média e pontuação média do time', trendline='ols')

fig5.show()

fig6 = px.box(players_score, y=["score_value", "score_per_round_value", "avg_score_value", "avg_team_score_value", "team_score_value"])

fig6.show()


A análise dos dados revela **números** significativos e **porcentagens** relevantes sobre as pontuações **individuais** e de **times** de Valorant.

Em relação à **pontuação individual**, observa-se uma **média** de **4.312 pontos**, representando aproximadamente **43.12%** do valor **máximo** registrado de **12.105k**. A pontuação **mediana** de **4.123** indica que **metade dos jogadores** obteve uma pontuação que corresponde a cerca de **41.23% do valor máximo**. Por outro lado, a pontuação **mínima** de **162** representa aproximadamente **1.34%** do **valor máximo**, evidênciando uma **discrepância significativa** entre os jogadores.

Ao analisar a **pontuação por round**, observa-se uma **média de 211.3006 pontos**, o que equivale aproximadamente a **35.37%** do **valor máximo** registrado de **595.6154**. A pontuação **mediana** de **203.3214** corresponde a cerca de **34.11%** do **valor máximo**, enquanto a pontuação **mínima de 20.26667** representa aproximadamente **3.40% do valor máximo**, demonstrando uma **disparidade considerável** nas pontuações alcançadas em diferentes rodadas.

Em vista à relação entre a **pontuação individual** e a **quantidade de pontos dividida pela quantidade de rounds**, a **média de 207.8606** equivale a cerca de **36.48% do valor máximo** registrado de **569.8125**. A pontuação **individual máxima**  é de **569.8125**, representando um **desempenho excepcional** que corresponde a **100% do valor máximo**. Por outro lado, a **pontuação individual mínima** de **12.46154** corresponde aproximadamente a **2.18% do valor máximo**, evidenciando uma **disparidade significativa** nas performances individuais.

No que diz respeito à **pontuação média do time**, a **média de 1.038.877 pontos** corresponde a aproximadamente **80.15% do valor máximo** registrado de **1.295.769**. A pontuação **mediana de 1.058.619** representa cerca de **81.73% do valor máximo**, enquanto a pontuação **mínima de 240.2308** equivale a aproximadamente **18.54%** do valor máximo, indicando uma **diferença considerável** na performance dos **times**.

Ao analisar a **pontuação por resultados de partidas**, a **média** da pontuação nas vitórias é de **208.1179**, enquanto nas **derrotas** é de **207.2034**. Esses valores representam aproximadamente **1.71%** e **1.70%**, respectivamente, em relação ao **valor máximo** registrado. A **mediana** da pontuação nas **vitórias é de 22.204**, correspondendo a cerca de **18.33%** do **valor máximo**, enquanto na **derrota** a **mediana** é de **21.5065**, representando aproximadamente **17.71%** do **valor máximo**. Quanto aos **valores máximos**, uma **vitória** registrou uma **pontuação máxima** de **33.88k**, equivalente a cerca de **27.90%** do **valor máximo**, enquanto uma **derrota** teve um **valor máximo** de **33.533k**, correspondendo a aproximadamente **27.58%** do **valor máximo**. As pontuações **mínimas** apresentam uma **diferença significativa**, com uma **vitória** mínima de **3.123**, que equivale a aproximadamente **0.26%** do **valor máximo**, e uma **derrota** mínima de **9.033**, representando cerca de **0.74%** do **valor máximo**.

##### KDA (Kills / Deaths / Assists)

In [382]:
player_kda = df_player_book_pd[['result', 'kills_value', 'kills_per_round_value', 'assists_value', 'kd_ratio_value', 'avg_first_kills_value', 'avg_first_deaths_value']]
player_kda.head(2)

,result,kills_value,kills_per_round_value,assists_value,kd_ratio_value,avg_first_kills_value,avg_first_deaths_value
0,victory,16,0.94,2,1.60,0.18,0.00
1,victory,7,0.41,2,0.44,0.00,0.24


In [399]:
numeric_cols = player_kda.drop('result', axis=1)  # Excluindo a coluna 'result'

min_values = numeric_cols.min()
max_values = numeric_cols.max()
median_values = numeric_cols.median()

fig = px.bar(
    pd.concat([min_values, max_values, median_values], axis=1),
    labels={'index': 'Métricas', 'value': 'Valores'},
    barmode='group',
    title='Valores Mínimos, Máximos e Medianas'
)
fig.show()

result_counts = player_kda['result'].value_counts()

fig = px.pie(
    result_counts,
    names=result_counts.index,
    title='Distribuição Percentual dos Resultados'
)
fig.show()

fig = px.scatter(
    player_kda,
    x='result',
    y='kills_value',
    title='Relação entre Performance e Resultado'
)
fig.show()

grouped_data = player_kda.groupby('result').mean()

fig = px.bar(
    grouped_data,
    labels={'index': 'Resultados', 'value': 'Valores Médios'},
    title='Valores Médios Agrupados por Resultados'
)
fig.show()

In [368]:
df_player_book_pd.head(2)

,match_id,platform_user_identifier,total_spent_credits,total_rounds,rank_value,current_rank_value,score_value,score_per_round_value,kills_per_round_value,kills_value,deaths_value,assists_value,kd_ratio_value,damage_value,damage_per_round_value,damage_delta_per_round_value,single_kills_value,double_kills_value,triple_kills_value,quadra_kills_value,penta_kills_value,multi_kills_value,grenade_casts_value,ability1_casts_value,ability2_casts_value,ultimate_casts_value,grenade_casts_per_round_value,ability1_casts_per_round_value,ability2_casts_per_round_value,ultimate_casts_per_round_value,plants_value,defuses_value,first_kills_value,first_deaths_value,esr_value,first_kills_per_round_value,first_deaths_per_round_value,econ_rating_value,hs_accuracy_value,kast_value,clutches_value,rounds_win_pct_value,trn_performance_score_value,party_id,team_id,agent_name,has_won,rounds_won_value,rounds_lost_value,team_score_value,team_kills_value,team_deaths_value,team_assists_value,team_damage_value,weapon_name,max_total_kills_weapon_name,is_available,timestamp,season_name,map_name,playtime_value,result,expiry_date,mode_key,mode_name,mode_max_rounds,duration,date_started,total_players_match,total_players_team,date_match,hour_match,minutes_match,playtime_minutes_value,playtime_hours_value,week_day_number,week_day,rank_number_value,current_rank_number_value,team_number_id,agent_name_number,result_number,weapon_name_number,is_available_number,map_name_number,mode_key_number,mode_name_number,avg_spent_credits,avg_score_value,avg_kills_value,avg_deaths_value,avg_assists_value,avg_damage_value,avg_grenade_casts_value,avg_ability1_casts_value,avg_ability2_casts_value,avg_ultimate_casts_value,avg_plants_value,avg_defuses_value,avg_first_kills_value,avg_first_deaths_value,avg_esr_value,avg_econ_rating_value,avg_kast_value,avg_clutches_value,avg_trn_performance_score_value,avg_team_score_value,avg_team_kills_value,avg_team_deaths_value,avg_team_assists_value,avg_team_damage_value
0,01f95d78-c404-4c9f-9a27-22b54f1fdf1c,Makitori#BR1,32550.00,17,Bronze 3,Silver 3,4282,251.88,0.94,16,10,2,1.60,2858,168.12,59.00,5,4,1,0,0,1,16.00,5.00,9.00,2.00,0.94,0.29,0.53,0.12,1,1,3,0,100.00,0.18,0.00,89,27.27,71.00,1,76.47,834,f6aa7b58-3bd2-4b33-8a02-56c2baa8a23f,Blue,Chamber,True,13,4,20989,75,49,27,13406,Vandal,8,True,2022-10-21T20:09:55.848+00:00,E5: A3,Ascent,1635,victory,2023-05-22T19:51:48.3816713+00:00,bomb,Competitive,25,1635054,2022-10-21T20:09:55.848+00:00,10,5,2022-10-21,20,9,27.25,0.45,6,Friday,7,10,2,17,2,2,1,1,1,1,1914,251.88,0.94,0.59,0.12,168.12,0.94,0.29,0.53,0.12,0.06,0.06,0.18,0.00,5.88,5.24,4.18,0.06,49.06,1234.65,4.41,2.88,1.59,788.59
1,01f95d78-c404-4c9f-9a27-22b54f1fdf1c,PiKaRoSaUrOReX#BR1,39250.00,17,Bronze 2,Silver 1,2284,134.35,0.41,7,16,2,0.44,1514,89.06,-80.65,5,1,0,0,0,0,21.00,7.00,4.00,1.00,1.24,0.41,0.24,0.06,3,0,0,4,0.00,0.00,0.24,38,8.70,53.00,0,23.53,3,7c594090-9c27-48a5-8358-ba32195dafa2,Red,Reyna,False,4,13,15783,49,75,16,10454,Vandal,2,True,2022-10-21T20:09:55.848+00:00,E5: A3,Ascent,1635,victory,2023-05-22T19:51:48.3816713+00:00,bomb,Competitive,25,1635054,2022-10-21T20:09:55.848+00:00,10,5,2022-10-21,20,9,27.25,0.45,6,Friday,6,8,1,6,2,2,1,1,1,1,2308,134.35,0.41,0.94,0.12,89.06,1.24,0.41,0.24,0.06,0.18,0.00,0.00,0.24,0.00,2.24,3.12,0.00,0.18,928.41,2.88,4.41,0.94,614.94
